In [1]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=1,name='make_profiles',walltime='00:20:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [2]:
#cluster.scale(10) 
cluster.adapt(minimum=1, maximum=20) 


In [3]:
from dask.distributed import Client
client = Client(cluster)


In [4]:
cluster

In [5]:


client



<Client: scheduler='tcp://172.30.100.3:37289' processes=0 cores=0>

In [6]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob

In [7]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'


In [8]:
import Box2x2 as bb


Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14


In [9]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthtv'}

def compute_mean_profile(var,date):
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file,chunks={'x':1000,'y':1000,'time_counter':1,filedeps[var]:1})
    data=ds[str(var)]
    for ibox in bb.boxes:
        box = ibox
        print(box.name)      
        profile=data[:,:,box.jmin:box.jmax,box.imin:box.imax].mean(dim={'x','y','time_counter'})
        profile.to_dataset().to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/eNATL60'+box.name+'-BLBT02_y'+date[0:4]+'m'+date[4:7]+'d'+date[6:9]+'_'+var+'-profile.nc',mode='w')


In [10]:
%time compute_mean_profile('votemper','20090701')

Box_1
Box_2
Box_3
Box_4
Box_5
Box_6
Box_7
Box_8
Box_9
Box_10
Box_11
Box_12
Box_13
Box_14
CPU times: user 4min 36s, sys: 19.7 s, total: 4min 55s
Wall time: 4min 58s


In [11]:
date='20090701'
print(date[0:4])
print(date[4:7])
print(date[6:9])

2009
070
01


/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/scheduler.py:313: UserWarning: WorkerState.ncores has moved to WorkerState.nthreads
  warnings.warn("WorkerState.ncores has moved to WorkerState.nthreads")
